In [ ]:
from matplotlib import cm
import piscola as pisco
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sn = pisco.call_sn('sn2004eo', directory='data_pantheon/')
for band in sn.bands:
    sn.data[band]['mag_sys'] = 'BD17'
    
sn.mask_data(snr=5)
sn.normalize_data()
sn.fit_lcs()

In [ ]:
new_palette = [plt.get_cmap('Dark2')(i) for i in np.arange(8)] + [plt.get_cmap('Set1')(i) for i in np.arange(8)]
band_list = sn.bands

fig, ax = plt.subplots(figsize=(8,6))
for i, band in enumerate(band_list):
    ylabel = 'Apparent Magnitude'
    mask = np.where(sn.data[band]['flux'] > 0)
    mjd = sn.data[band]['mjd'][mask] - 53270
    mag = -2.5*np.log10(sn.data[band]['flux'][mask]) + sn.data[band]['zp']
    err = np.abs(2.5*sn.data[band]['flux_err'][mask]/(sn.data[band]['flux'][mask]*np.log(10)))

    ax.errorbar(mjd[::4], mag[::4], err[::4], fmt='o', capsize=3, capthick=2, ms=24, mec='k', elinewidth=3, label=band, color=new_palette[i])

fig.patch.set_visible(False)
ax.axis('off')

fig.tight_layout()
plt.xlim(-5, 40)
plt.ylim(15, 20)
plt.gca().invert_yaxis()
plt.savefig(f'snia_lc.png')

plt.show()

In [ ]:
new_palette = [plt.get_cmap('Dark2')(i) for i in np.arange(8)] + [plt.get_cmap('Set1')(i) for i in np.arange(8)]


fig, ax = plt.subplots(figsize=(8, 6))
for i, band in enumerate(sn.bands):
    
    time, flux, std = np.copy(sn.lc_fits[band]['mjd']), np.copy(sn.lc_fits[band]['flux']), np.copy(sn.lc_fits[band]['std'])
    data_time, data_flux, data_std = np.copy(sn.data[band]['mjd']), np.copy(sn.data[band]['flux']), np.copy(sn.data[band]['flux_err'])

    # avoid non-positive numbers in logarithm
    fit_mask = flux > 0
    time, flux, std = time[fit_mask] - 53270, flux[fit_mask], std[fit_mask]
    data_mask = data_flux > 0
    data_time, data_flux, data_std = data_time[data_mask] - 53270, data_flux[data_mask], data_std[data_mask]

    mag = -2.5*np.log10(flux) + sn.data[band]['zp']
    err = np.abs(2.5*std/(flux*np.log(10)))
    data_mag = -2.5*np.log10(data_flux) + sn.data[band]['zp']
    data_err = np.abs(2.5*data_std/(data_flux*np.log(10)))

    ax.errorbar(data_time[::4], data_mag[::4], data_err[::4], fmt='o', mec='k', capsize=3, capthick=2, ms=24,
                elinewidth=3, color=new_palette[i],label=band)
    ax.plot(time, mag,'-', color=new_palette[i], lw=10, zorder=16)
    ax.fill_between(time, mag-err, mag+err, alpha=0.5, color=new_palette[i])

fig.patch.set_visible(False)
ax.axis('off')

plt.gca().invert_yaxis()
fig.tight_layout()
plt.xlim(-5, 40)
plt.ylim(15, 20)
plt.gca().invert_yaxis()
plt.savefig(f'lc_fits.png')

plt.show()

In [ ]:
df = sn.sed['info'][sn.sed['info'].phase==0]
x, y = df.wave, df.flux

fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(x, y, lw=18, c='k')
for i, band in enumerate(sn.bands):
    filt_wave, filt_trans = sn.filters[band]['wave'], sn.filters[band]['transmission']
    eff_wave = sn.filters[band]['eff_wave']
    flux = pisco.filter_utils.integrate_filter(x, y, filt_wave, filt_trans)
    
    ax.errorbar(eff_wave, flux, yerr=0.0, fmt='o', mec='k', capsize=3, capthick=2, ms=45,
                elinewidth=3, color=new_palette[i])
    
fig.patch.set_visible(False)
ax.axis('off')

fig.tight_layout()
plt.xlim(2000, 9000)
path = 'anaconda3/envs/pisco/lib/python3.6/site-packages/resources/piscola/lightcurves'
plt.savefig(f'sed_match.png')
plt.show()

In [ ]:
cmap = cm.get_cmap('viridis')

z_range = np.linspace(0, 0.5, 5)
df = sn.sed['info'][sn.sed['info'].phase==0]
x, y = df.wave, df.flux

fig, ax = plt.subplots(figsize=(12, 8))
for i, z in enumerate(z_range):
    ax.plot(x*(1+z), y/(1+z), lw=18, c=cmap(z*2))
    
fig.patch.set_visible(False)
ax.axis('off')

fig.tight_layout()
plt.xlim(2000, 12000)
plt.savefig(f'kcorr.png')
plt.show()